In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json("google_news_dfset.json")

In [3]:
df.sample(10)

In [4]:
df.info()

<class 'pandas.core.frame.dfFrame'>
Index: 1517 entries, 0 to 1516
df columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Source    1517 non-null   object
 1   Headline  1517 non-null   object
 2   Category  1517 non-null   object
dtypes: object(3)
memory usage: 47.4+ KB


In [5]:
import seaborn as sns

In [6]:
frequency = []

for x in list(df.groupby('Source')):
    
    frequency.append([x[0],len(x[1])])

In [7]:
frequency.sort(key=lambda x: x[1])

In [8]:
frequency[-5:]

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import json
import os

In [10]:
df = df.sample(frac=1)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [12]:
X_train, X_val, y_train, y_val = train_test_split(df[['Headline']], df['Category'], test_size=0.3, random_state=42)

In [13]:
le = LabelEncoder()
enc = OneHotEncoder(sparse_output=False)
le.fit(df['Category'])
enc.fit(le.transform(y_train).reshape(-1, 1))
y_train = enc.transform(le.transform(y_train).reshape(-1, 1))
y_val = enc.transform(le.transform(y_val).reshape(-1, 1))
le.classes_

In [14]:
dir = "\\".join(os.getcwd().strip().split('\\')[:-1])
dir

In [15]:

bert_tokenizer = hub.KerasLayer(dir+"\\Tensorflow_Hub_Models\\bert_en_uncased_L-12_H-768_A-12_4")
bert_preprocessor = hub.KerasLayer(dir+"\\Tensorflow_Hub_Models\\bert_en_uncased_preprocess_3")

In [16]:
text_input = tf.keras.layers.Input(shape=(),dtype=tf.string)

text_preprocessed = bert_preprocessor(text_input)

outputs = bert_tokenizer(text_preprocessed)

hidden1 = tf.keras.layers.Dense(units=768,activation='relu')(outputs['sequence_output'])

hidden2 = tf.keras.layers.LSTM(units=768,activation='tanh')(hidden1)

hidden3 = tf.keras.layers.Dropout(0.3)(hidden2)

hidden4 = tf.keras.layers.Dense(len(df['Category'].unique()), activation='softmax')(hidden3)

model = tf.keras.Model(inputs=[text_input],outputs=[hidden4])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     {'input_mask': (Non  0           ['input_1[0][0]']                
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [17]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.categorical_crossentropy,
  metrics=['accuracy'],
)

In [18]:
with tf.device('/GPU:0'):
    model.fit(x=X_train, y=y_train,epochs=10,validation_df=(X_val,y_val), batch_size=16)

Epoch 1/10
67/67 [==============================] - 23s 185ms/step - loss: 0.8276 - accuracy: 0.7107 - val_loss: 0.6349 - val_accuracy: 0.7807
Epoch 2/10
67/67 [==============================] - 11s 169ms/step - loss: 0.3875 - accuracy: 0.8775 - val_loss: 0.7209 - val_accuracy: 0.7741
Epoch 3/10
67/67 [==============================] - 11s 169ms/step - loss: 0.3162 - accuracy: 0.8935 - val_loss: 0.5719 - val_accuracy: 0.8377
Epoch 4/10
67/67 [==============================] - 11s 169ms/step - loss: 0.1713 - accuracy: 0.9434 - val_loss: 0.5600 - val_accuracy: 0.8377
Epoch 5/10
67/67 [==============================] - 11s 169ms/step - loss: 0.1431 - accuracy: 0.9463 - val_loss: 0.5064 - val_accuracy: 0.8509
Epoch 6/10
67/67 [==============================] - 11s 169ms/step - loss: 0.0800 - accuracy: 0.9736 - val_loss: 0.6951 - val_accuracy: 0.8224
Epoch 7/10
67/67 [==============================] - 11s 170ms/step - loss: 0.1292 - accuracy: 0.9529 - val_loss: 0.5545 - val_accuracy: 0.8355

In [19]:
model.save('news_headlines.keras')

In [20]:
X = df.sample(100)
X_test = X[['Headline']]
y_test = enc.transform(le.transform(X['Category']).reshape(-1, 1)) 

In [21]:
model.evaluate(X_test,y_test)

4/4 [==============================] - 1s 160ms/step - loss: 0.3066 - accuracy: 0.9400


In [22]:
prediction = model.predict(['MEA sets up 24-hour control room in view of Israel-Palestine conflict'])

1/1 [==============================] - 1s 940ms/step


In [23]:
for p in prediction:
    
    print(le.classes_[p.argmax()])

politics
